# Navier Stokes - flow around cylinder

## Formulation

In this [benchmark example](https://wwwold.mathematik.tu-dortmund.de/~featflow/en/benchmarks/cfdbenchmarking/flow/dfg_benchmark3_re100.html) we will show how to compute the flow of a Newtonian fluid around a cylinder. Even though the inlet velocity profile is constant, the resulting velocity field may not be stationary. We will see a pattern called [Kármán vortex street](https://en.wikipedia.org/wiki/K%C3%A1rm%C3%A1n_vortex_street).

The following figure shows the computational mesh used. 
![Mesh](fig/hron_turek_bench.png "Mesh")
The strong formulation of the time dependent Navier Stokes equations for this problem is as follows
\begin{align}
\frac{\partial\mathbf{u}}{\partial t} + \mathbf{u}\cdot\nabla\mathbf{u} + \nabla p - \nu\Delta\mathbf{u} &= 0\quad\text{in }\,\Omega\times (0,T)\\
\nabla\cdot\mathbf{u}&=0\quad\text{in }\,\Omega\times (0,T)\\
\mathbf{u}(t=0)&=\mathbf{0}\quad\text{in }\,\Omega\\
\mathbf{u} &= \mathbf{u}_{\text{in}}\quad\text{on }\,\Gamma_{\text{in}}\times (0,T)\\
\mathbf{u} &= \mathbf{0}\quad\text{on }\,\Gamma_{\text{wall}}\times (0,T)\\
\mathbf{u} &= \mathbf{0}\quad\text{on }\,\Gamma_{\text{cylinder}}\times (0,T)\\
(-p \mathbb{I} + \nu \nabla \mathbf{u}) \mathbf{n} &= \mathbf{0}\quad\text{on }\,\Gamma_{\text{out}}\times (0,T),
\end{align}
where $\mathbf{u}_{\text{in}}$ is defined as 
$$
\mathbf{u}_{\text{in}} = v_{\text{avg}}\begin{pmatrix}
\frac{4\,y\,(D-y)}{D^2}\\
0
\end{pmatrix}
$$

Taking test functions $q$ and $\mathbf{v}$ for pressure and velocity respectively, we multiply the first equation by $\mathbf{v}$ and the second equation by $q$ and integrate over $\Omega$. Then we use the integration by parts to obtain the weak formulation.

$$
\int_\Omega\frac{\partial\mathbf{u}}{\partial t}\cdot\mathbf{v}\,\text{d}x+\int_\Omega(\mathbf{u}\cdot\nabla\mathbf{u})\cdot\mathbf{v}\,\text{d}x+\nu\int_\Omega\nabla\mathbf{u}\cdot\nabla\mathbf{v}\,\text{d}x-\int_\Omega p\,\text{div}\,\mathbf{v}\,\text{d}x-\int_\Omega q\,\text{div}\,\mathbf{u}\,\text{d}x = 0
$$

Compared to the Stokes equations, the Navier Stokes equations include a convection term $\int_\Omega(\mathbf{u}\cdot\nabla\mathbf{u})\cdot\mathbf{v}\,\text{d}x$. Therefore, a nonlinear solver has to be used instead of a linear solver. 

Another new term is the time derivative $\frac{\partial \mathbf{u}}{\partial t}$ which we will treat using a Crank-Nicholson time stepping scheme.
First we discretize the time derivative.
\begin{equation}
\frac{\partial \mathbf{u}}{\partial t}(t) \approx \frac{\mathbf{u}(t)-\mathbf{u}(t-\delta t)}{\delta t}
\end{equation}
In order to use the Crank-Nicolson scheme, we make a linear combination of explicit and implicit scheme using a factor $\theta$ in part of the equation (here we use simpler notation $\mathbf{u}^{i} = \mathbf{u}(t)$, $\mathbf{u}^{i-1} = \mathbf{u}(t-\delta t)$). Notice that the last two terms are not part of the linear combination since we need to use the implicit scheme on them to have stability.
\begin{equation}
\begin{split}
\int_\Omega\frac{\mathbf{u}^i-\mathbf{u}^{i-1}}{\delta t}\cdot\mathbf{v}\,\text{d}x\\
+\theta\left(\int_\Omega(\mathbf{u}^i\cdot\nabla\mathbf{u}^i)\cdot\mathbf{v}\,\text{d}x+\nu\int_\Omega\nabla\mathbf{u}^i\cdot\nabla\mathbf{v}\,\text{d}x\right)\\+(1-\theta)\left(\int_\Omega(\mathbf{u}^{i-1}\cdot\nabla\mathbf{u}^{i-1})\cdot\mathbf{v}\,\text{d}x\\+\nu\int_\Omega\nabla\mathbf{u}^{i-1}\cdot\nabla\mathbf{v}\,\text{d}x\right)\\
-\int_\Omega p^i\,\text{div}\,\mathbf{v}\,\text{d}x-\int_\Omega q\,\text{div}\,\mathbf{u}^i\,\text{d}x = 0
\end{split}
\end{equation}

## Implementation
First, we need to import all neccessary packages.

In [1]:
import dolfin as df
from time import time

Then we read mesh and boundary markers from the file.

In [12]:
mesh = df.Mesh()

# read mesh from hdf5 file

# read boundary marks

# save boundary marks as xdmf file; then, we can open it in ParaView


Next, we define the function spaces V, P and create the mixed function space W. The [MINI](https://defelement.com/elements/mini.html) element is used in this example.

<font size="4"> <b> Task 1</b>: Finish implementation of MINI element. </font>

In [3]:
dim = mesh.geometry().dim()
U = ...
B = ...
P = ...
V = df.VectorElement(df.NodalEnrichedElement(U, B))
W = df.FunctionSpace(mesh, df.MixedElement([V, P]))

Let us define some parameters and constants such are the average velocity at the inlet, kinematic viscosity, timestep size and time T.

In [19]:
velocity = 1.5
nu = df.Constant(0.001)
dt = 0.1
t_end = 15

Let us define the boundary conditions. The boundary markings are inlet: 1, outlet: 2, wall: 3, cylinder: 5. We prescribe zero vector on the wall and the cylinder using Dirichlet boundary conditions. Inlet velocity can be defined using an expression object.

<font size="4"> <b> Task 2</b>: Complete the expression for inlet velocity according to the formula above. </font>

In [5]:
inlet_expression = ...
bc_in = ...

zero_vector = ...
bc_walls = df.DirichletBC(W.sub(0), zero_vector, bndry, 3)
bc_sphere = df.DirichletBC(W.sub(0), zero_vector, bndry, 5)

bcs = ...

Let's see the results for Stokes flow.

<font size="4"> <b> Task 2</b>: Run the code below (Stokes flow) for maximum inlet velocity=1.5 and/or 0.3 and visualize results in ParaView. </font>

In [20]:
# Stokes example we did on Tuesday
u, p = df.TrialFunctions(W)
v, q = df.TestFunctions(W)
F = nu*df.inner(df.grad(u), df.grad(v))*df.dx - p*df.div(v)*df.dx - q*df.div(u)*df.dx

w = df.Function(W)
problem = df.LinearVariationalProblem(df.lhs(F), df.rhs(F), w, bcs)
solver = df.LinearVariationalSolver(problem)
solver.parameters["linear_solver"] = "mumps"

tick = time()
solver.solve()

print("ellapsed = ", time() - tick, "s")

ufile = df.XDMFFile("results_stokes/u.xdmf")
pfile = df.XDMFFile("results_stokes/p.xdmf")
u, p = w.split()
u.rename("u", "velocity")
p.rename("p", "pressure")
ufile.write(u)
pfile.write(p)

ellapsed =  0.416898250579834 s


Now we define the variational form for functions $\mathbf{u}, p$ with help of test functions $\mathbf{v}, q$ and functions $\mathbf{u}_0, p_0$ representing the velocity and pressure fields from previous timestep.

<font size="4"> <b> Task 3</b>: Add time stepping to our variational formulation. </font>

In [6]:
# define v and q as test functions

# define w and split to u and p

# do the same for w0 -> u0, p0

# we will use the following definions to simplify the notation a bit
# NOTE that we are still working with Stokes flow
a = lambda u, v: nu*df.inner(df.grad(u), df.grad(v))*df.dx
b = lambda q, v: q*df.div(v)*df.dx

# define F0 and F1

theta = df.Constant(0.5) # Crank–Nicolson scheme

# define F

# Jacobi matrix


We create a nonlinear variational problem object using our variational form and a nonlinear variational solver. Optionally, we can set up some of the solver parameters such as absolute and relative tolerance for convergence.

In [27]:
# define Nonlinear variational problem and solver and specify some parameters

solver.parameters['newton_solver']['linear_solver'] = 'mumps'
solver.parameters['newton_solver']['absolute_tolerance'] = 1e-12
solver.parameters['newton_solver']['relative_tolerance'] = 1e-12

Setup XDMF files to save results for viewing in Paraview.

In [28]:
ufile = df.XDMFFile("results/u.xdmf")
pfile = df.XDMFFile("results/p.xdmf")
ufile.parameters["flush_output"] = True
pfile.parameters["flush_output"] = True

Since we have time dependent problem, we need to solve the problem for each timestep using a loop. We start at time $t=0$, then for each timestep, we assign the previous solution $w$ to $w_0$ and solve for the new $w$. It is also a good idea to save the solution at each timestep into the XDMF file.

<font size="4"> <b> Task 4</b>: Run the simulation for velocity=0.3 and/or 1.5 for Stokes flow and compare results with the previous case. You can try different time stepping schemes. </font> Additional task: Can you calculate the difference between two velocity fields using ParaView?

<font size="4"> <b> Task 5</b>: Add the convective term into form 'a' (Navier-Stokes) and run the simulation for velocity=0.3 and 1.5. Which result show Kármán vortex street? </font>

In [30]:
tick = time()
t = 0.0
(u, p) = w.split(deepcopy=True)
u.rename("v", "velocity")
p.rename("p", "pressure")
df.assign(u, w.sub(0))
df.assign(p, w.sub(1))
ufile.write(u, t)
pfile.write(p, t)
while t < t_end:
    w0.assign(w)
    t += dt
    print("t={:.1f}s".format(t))
    solver.solve()
    df.assign(u, w.sub(0))
    df.assign(p, w.sub(1))
    ufile.write(u, t)
    pfile.write(p, t)
print("ellapsed = ", time() - tick, "s")

t=0.10s
t=0.20s
t=0.30s
t=0.40s
t=0.50s
t=0.60s
t=0.70s
t=0.80s
t=0.90s
t=1.00s
t=1.10s
t=1.20s
t=1.30s
t=1.40s
t=1.50s
t=1.60s
t=1.70s
t=1.80s
t=1.90s
t=2.00s
t=2.10s
t=2.20s
t=2.30s
t=2.40s
t=2.50s
t=2.60s
t=2.70s
t=2.80s
t=2.90s
t=3.00s
t=3.10s
t=3.20s
t=3.30s
t=3.40s
t=3.50s
t=3.60s
t=3.70s
t=3.80s
t=3.90s
t=4.00s
t=4.10s
t=4.20s
t=4.30s
t=4.40s
t=4.50s
t=4.60s
t=4.70s
t=4.80s
t=4.90s
t=5.00s
t=5.10s
t=5.20s
t=5.30s
t=5.40s
t=5.50s
t=5.60s
t=5.70s
t=5.80s
t=5.90s
t=6.00s
t=6.10s
t=6.20s
t=6.30s
t=6.40s
t=6.50s
t=6.60s
t=6.70s
t=6.80s
t=6.90s
t=7.00s
t=7.10s
t=7.20s
t=7.30s
t=7.40s
t=7.50s
t=7.60s
t=7.70s
t=7.80s
t=7.90s
t=8.00s
t=8.10s
t=8.20s
t=8.30s
t=8.40s
t=8.50s
t=8.60s
t=8.70s
t=8.80s
t=8.90s
t=9.00s
t=9.10s
t=9.20s
t=9.30s
t=9.40s
t=9.50s
t=9.60s
t=9.70s
t=9.80s
t=9.90s
t=10.00s
t=10.10s
t=10.20s
t=10.30s
t=10.40s
t=10.50s
t=10.60s
t=10.70s
t=10.80s
t=10.90s
t=11.00s
t=11.10s
t=11.20s
t=11.30s
t=11.40s
t=11.50s
t=11.60s
t=11.70s
t=11.80s
t=11.90s
t=12.00s
t=12.10s
t=12.20s
t